<a href="https://colab.research.google.com/github/Astharuth-Villa/INDICE_DE_PRECIPITACION-EXTREMA/blob/main/IOA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%%capture
!pip install netcdf4
!pip install geopandas
!pip install rioxarray
!pip install xclim
!pip install rasterio

In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from geopandas import read_file as gpd_read_file
import pylab as pl
import seaborn as sns
import zipfile
import geopandas as gpd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [5]:
#THE PACIFIC DECADAL OSCILLATION INDEX (PDOi)
PDO = pd.read_csv('https://psl.noaa.gov/data/correlation/pdo.data', sep='\s+', skiprows=1, skipfooter = 14, engine='python', header=None, na_values=-9.90, index_col=0)
PDO_year=PDO.mean(1)
PDO_filtro = PDO_year.loc["1981":"2016"]
PDO_filtro

0
1981    0.918333
1982    0.114167
1983    1.648333
1984    0.837500
1985    0.449167
1986    1.239167
1987    1.820833
1988    0.531667
1989   -0.179167
1990   -0.355833
1991   -0.419167
1992    0.928333
1993    1.416667
1994   -0.151667
1995    0.642500
1996    0.640833
1997    1.460833
1998    0.245833
1999   -1.063333
2000   -0.590000
2001   -0.562500
2002    0.220833
2003    0.969167
2004    0.345000
2005    0.375000
2006    0.190833
2007   -0.195833
2008   -1.292500
2009   -0.612500
2010   -0.312500
2011   -1.230833
2012   -1.100000
2013   -0.517500
2014    1.130833
2015    1.634167
2016    1.542500
dtype: float64

In [6]:
#THE EASTERN INDEX (index E)
tabla = pd.read_csv("/content/drive/MyDrive/PROGRAMACIÓN_2/DATA/ecindex_ersstv5_CSV.csv")
tabla['month']=tabla['month'].astype(str)
tabla['year']=tabla['year'].astype(str)
tabla["FECHA"]=tabla.year.str.cat(tabla.month,sep="-")
tabla["FECHA"]=pd.to_datetime(tabla["FECHA"])
tabla=tabla.set_index("FECHA")
tabla=tabla.drop(columns=["year","month"],axis=1)

In [7]:
tabla_E = tabla['E_index'].resample("Y").sum()
tabla_E=tabla_E.loc["1981-12-31":"2016-12-31"]
tabla_E

FECHA
1981-12-31    -5.52801
1982-12-31    11.29164
1983-12-31    27.49673
1984-12-31    -1.83103
1985-12-31    -9.69659
1986-12-31    -2.52522
1987-12-31    13.88367
1988-12-31   -10.33723
1989-12-31    -2.35616
1990-12-31    -5.90980
1991-12-31     0.27125
1992-12-31     4.32776
1993-12-31     5.82076
1994-12-31    -5.06080
1995-12-31    -9.20491
1996-12-31    -9.62179
1997-12-31    24.19572
1998-12-31    22.89806
1999-12-31    -1.28809
2000-12-31    -0.22549
2001-12-31    -4.47818
2002-12-31     0.90465
2003-12-31    -4.36658
2004-12-31    -4.49157
2005-12-31    -7.88823
2006-12-31     2.20899
2007-12-31   -11.14157
2008-12-31     8.14758
2009-12-31     3.18636
2010-12-31    -4.48054
2011-12-31    -0.95147
2012-12-31     4.84952
2013-12-31    -8.80650
2014-12-31     0.79893
2015-12-31    13.31732
2016-12-31     3.56155
Freq: A-DEC, Name: E_index, dtype: float64

In [8]:
#THE CENTRAL INDEX (index C)
tabla_C = tabla['C_index'].resample("Y").sum()
tabla_C=tabla_C.loc["1981-12-31":"2016-12-31"]
tabla_C

FECHA
1981-12-31    -5.91708
1982-12-31     4.68182
1983-12-31   -11.14535
1984-12-31   -11.79383
1985-12-31    -9.06367
1986-12-31     1.02940
1987-12-31     7.22661
1988-12-31    -9.10022
1989-12-31   -13.79241
1990-12-31     5.84995
1991-12-31    10.01285
1992-12-31     6.36460
1993-12-31     2.35216
1994-12-31     9.09695
1995-12-31     3.05537
1996-12-31    -4.39944
1997-12-31     2.19741
1998-12-31   -15.13197
1999-12-31   -21.92176
2000-12-31   -17.17976
2001-12-31    -3.81540
2002-12-31     8.30668
2003-12-31     7.19082
2004-12-31     9.01808
2005-12-31     5.04171
2006-12-31     0.62358
2007-12-31    -2.69283
2008-12-31   -19.54408
2009-12-31     2.88932
2010-12-31    -6.41499
2011-12-31   -15.95063
2012-12-31    -6.32056
2013-12-31    -0.51171
2014-12-31     4.36319
2015-12-31    16.22703
2016-12-31     5.58685
Freq: A-DEC, Name: C_index, dtype: float64

In [9]:
#THE TROPICAL NORTH ATLANTIC INDEX (TNAi)
TNA = pd.read_csv('https://psl.noaa.gov/data/correlation/tna.data', sep='\s+', skiprows=1, skipfooter = 6, engine='python', header=None, na_values=-99.99, index_col=0)
TNA_year=TNA.mean(1)
TNA_filtro = TNA_year.loc["1981":"2016"]
TNA_filtro

0
1981    0.227500
1982   -0.203333
1983    0.101667
1984   -0.282500
1985   -0.319167
1986   -0.489167
1987    0.318333
1988    0.092500
1989   -0.190000
1990    0.135000
1991   -0.247500
1992   -0.146667
1993   -0.170833
1994   -0.362500
1995    0.305833
1996    0.225000
1997    0.367500
1998    0.561667
1999    0.095000
2000   -0.048333
2001    0.260833
2002    0.173333
2003    0.330833
2004    0.481667
2005    0.671667
2006    0.470000
2007    0.241667
2008    0.315000
2009    0.146667
2010    1.013333
2011    0.450000
2012    0.310833
2013    0.482500
2014    0.125833
2015    0.257500
2016    0.434167
dtype: float64

In [10]:
#THE TROPICAL SOUTH ATLANTIC INDEX (TSAi)
TSA = pd.read_csv('https://psl.noaa.gov/data/correlation/tsa.data', sep='\s+', skiprows=1, skipfooter = 4, engine='python', header=None, na_values=-99.99, index_col=0)
TSA_year=TSA.mean(1)
TSA_filtro = TSA_year.loc["1981":"2016"]
TSA_filtro

0
1981   -0.143333
1982   -0.285833
1983   -0.047500
1984    0.554167
1985    0.216667
1986    0.224167
1987    0.307500
1988    0.504167
1989    0.229167
1990    0.055000
1991    0.011667
1992   -0.537500
1993    0.183333
1994    0.036667
1995    0.325000
1996    0.390833
1997   -0.031667
1998    0.531667
1999    0.365000
2000    0.199167
2001    0.216667
2002    0.193333
2003    0.472500
2004    0.133333
2005    0.099167
2006    0.264167
2007    0.208333
2008    0.479167
2009    0.459167
2010    0.588333
2011    0.267500
2012   -0.090833
2013    0.216667
2014    0.176667
2015    0.295833
2016    0.520000
dtype: float64

In [11]:
#TN-TSi
TNS = TNA_filtro-TSA_filtro
TNS

0
1981    0.370833
1982    0.082500
1983    0.149167
1984   -0.836667
1985   -0.535833
1986   -0.713333
1987    0.010833
1988   -0.411667
1989   -0.419167
1990    0.080000
1991   -0.259167
1992    0.390833
1993   -0.354167
1994   -0.399167
1995   -0.019167
1996   -0.165833
1997    0.399167
1998    0.030000
1999   -0.270000
2000   -0.247500
2001    0.044167
2002   -0.020000
2003   -0.141667
2004    0.348333
2005    0.572500
2006    0.205833
2007    0.033333
2008   -0.164167
2009   -0.312500
2010    0.425000
2011    0.182500
2012    0.401667
2013    0.265833
2014   -0.050833
2015   -0.038333
2016   -0.085833
dtype: float64